## some config

In [1]:
import os
import math
import random
import torch
import fire
import copy
import multiprocessing as mp
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.cuda.amp import GradScaler
# import deepspeed
from datetime import datetime
from typing import Optional
from pprint import pprint, pformat
from torch.utils.data import DataLoader, IterableDataset
from timm.utils import ModelEmaV3
from timm.models import load_checkpoint
from timm.utils.model import unwrap_model, get_state_dict
try:
    from torch.utils.tensorboard import SummaryWriter
except ModuleNotFoundError:
    from tensorboardX import SummaryWriter

import sys

# sys.path.insert(0, "..")
sys.path.insert(0, ".")

In [2]:
from src.data import (
    collator,
    vocab_builder,
    tokenizer,
    read_dataset,
    OdpsTableIterableDataset,
)
from src.models import (
    GraphGPTConfig,
    GraphGPTCausal,
    GraphGPT2Config,
    GraphGPT2Causal,
    GraphBertConfig,
    GraphBertForMaskedLM,
)
from src.utils import (
    conf_utils,
    loss_utils,
    loader_utils,
    tokenizer_utils,
    modules_utils,
    misc_utils,
    print_trainable_parameters,
    print_params,
    inspect_tokenization_results,
    set_up_shuffle_and_sampler,
    worker_init_fn_seed,
)

dict_models = {
    "graphgpt2": (GraphGPT2Causal, GraphGPT2Config),
    "graphgpt": (GraphGPTCausal, GraphGPTConfig),
    "graphbert": (GraphBertForMaskedLM, GraphBertConfig),
}

[2024-12-21 02:24:42,234] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/yang/miniconda3/envs/graph_gpt/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/yang/miniconda3/envs/graph_gpt/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlvsym'
/home/yang/miniconda3/envs/graph_gpt/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/home/yang/miniconda3/envs/graph_gpt/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/home/yang/miniconda3/envs/graph_gpt/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/home/yang/miniconda3/envs/graph_gpt/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
/home/yang/miniconda3/envs/graph_gpt/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `shm_open'
/home/yang/miniconda3/envs/graph_gpt/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined

In [3]:
data_dir: str = "../data/OGB"
tables: str = ""
# deepspeed_config = "./examples/ds_config2_pt.json"
intermediate_size = 0
num_attention_heads = 0
hidden_size = 512
num_hidden_layers = 8
task_type='pretrain'
causal_attention = 1
lr=3e-4
model_type = 'graphgpt'
output_dir='./exp/models/pcqm4m-v2/test'
pretrain_cpt = '/datalake/datastore1/yang/graph-gpt/exp/models/pcqm4m-v2/medium_ntp/pt_ns_h512_l8_b8192_mpe1024_tk1e9_gelu_pretrain3.3m_nmlm_mrlinear_mtp0.8_0_0.2_lr3e-4_adp0.1_pdp0_edp0_mdp0_lsi0_short_gated_wd0.1'
samples_per_saving=1000000

batch_size = 1024
stack_method = 'short'

pack_tokens = 0
max_position_embeddings = 1024

task_type='pretrain'
total_tokens=1e9
batch_size = 1024
warmup_tokens=1e8

In [4]:
use_tb_writer = False           # use tensorboard writer
use_ema = False # False # use exponential moving average to smooth model
use_deepspeed = False # True # use deepspeed for training, good to set scheduler
if (intermediate_size == 0) and (num_attention_heads == 0): # True
    (
        hidden_size,
        intermediate_size,
        num_attention_heads,
        num_hidden_layers,
    ) = modules_utils.set_up_model_architect(
        hidden_size=hidden_size, num_hidden_layers=num_hidden_layers # 768 24 related to model names intermediate_size = hidden_size * 4, num_attention_heads = hidden_size // 64
    )# 768 3072 12 24
causal_attention = 0 if task_type == "pretrain-mlm" else causal_attention
print('hidden_size:', hidden_size, 'intermediate_size:', intermediate_size, 'num_attention_heads:', num_attention_heads, 'num_hidden_layers:', num_hidden_layers, 'causal_attention:', causal_attention) # 768 3072 12 24 1


# #########################
# betas = (0.9, 0.95) # used in AdamW optimizer, important for config beta
# #########################
# # lr * 0.1 -> from llama2 pre-train settings
# min_lr = lr * 0.1 if use_deepspeed else 0    # used in scheduler, when not using deepspeed.
# #########################
gpu_name = torch.cuda.get_device_name()
GraphModel, GraphModelConfig = dict_models[model_type] # Not instantiate yet
print('gpu_name:', gpu_name, 'GraphModel:', GraphModel, 'GraphModelConfig:', GraphModelConfig) 

if os.path.exists(os.path.join(output_dir, "log.csv")):
    print(
        f"log file {os.path.join(output_dir, 'log.csv')} exists, resume training from {output_dir} instead of initializing from pre-train ckp {pretrain_cpt}!"
    )
    pretrain_cpt = output_dir


# # 0. init distributed train and get gpu/device info
# dist.init_process_group(backend="nccl", init_method="env://")  # for distributed training
# dist.barrier() # for sync training
# world_size = dist.get_world_size() # 1 # number of GPUs
# rank = dist.get_rank() # 0 # current GPU index
# local_rank = os.environ.get("LOCAL_RANK") # 0 # current GPU index local to the node
# print(f"\nworld size: {world_size}, rank: {rank}, local rank: {local_rank}") # 1 0 0
# rnd_seed = torch.random.initial_seed() - rank
# random.seed(rnd_seed)
# print(f"seed random with {rnd_seed}") # 1234
# steps_per_saving = samples_per_saving // (world_size * batch_size) # 1000000 // (1 * 1024) = 976
# print(f"\nsteps_per_saving: {steps_per_saving}") # 976
# params = print_params(**locals())

hidden_size: 512 intermediate_size: 2048 num_attention_heads: 8 num_hidden_layers: 8 causal_attention: 1
gpu_name: NVIDIA RTX A6000 GraphModel: <class 'src.models.graphgpt.modeling_graphgpt.GraphGPTCausal'> GraphModelConfig: <class 'src.models.graphgpt.configuration_graphgpt.GraphGPTConfig'>


## load data

In [5]:
# tokenizer config loading
import json

# Load the JSON file
file_path = "./zhang_test/tokenizer_config.json"
with open(file_path, "r") as json_file:
    tokenizer_config = json.load(json_file)

# Print the loaded data
# pprint(tokenizer_config)

In [6]:
# 1.1 read configuration
assert "pretrain" in tokenizer_config["task_type"]
assert (
    tokenizer_config["semantics"]["attr_assignment"]   # first
    in tokenizer_utils.ATTR_ASSIGNMENT_TYPES   # ATTR_ASSIGNMENT_TYPES = {"first", "last", "random", "all", "mix"}
)
# pprint(tokenizer_config)
if tokenizer_config["tokenizer_class"] == "StackedGSTTokenizer":
    attr_dim = (
        tokenizer_config["semantics"]["edge"]["dim"] # 3
        + tokenizer_config["semantics"]["node"]["dim"] # 9
    ) # 12
    assert stack_method in ("short", "long", None), f"stack_method: {stack_method}" # short
    if tokenizer_config["structure"]["edge"]["remove_edge_type_token"]: # True
        stacked_feat = 1 + attr_dim
    else:
        stacked_feat = 2 + attr_dim
    next_n_token = stacked_feat
else:
    stacked_feat = 1
    next_n_token = 1 # maybe how many pack of tokens to predict
embed_dim = tokenizer_config["semantics"]["node"].get(
    "embed_dim", 0
) + tokenizer_config["semantics"]["edge"].get("embed_dim", 0) # 0
print(
    f"stacked_feat: {stacked_feat}, next_n_token: {next_n_token}, embed_dim: {embed_dim}" # 13 13 0
)

stacked_feat: 13, next_n_token: 13, embed_dim: 0


## build vocabulary

In [7]:
add_eos = False
rank = 0
stack_method = "short"
# 1.3 build vocab and then init tokenizer from the tokenization config
tokenizer_cls = getattr(tokenizer, tokenizer_config["tokenizer_class"]) # StackGSTTokenizer, custom defined
gtokenizer = tokenizer_cls(
    tokenizer_config, add_eos=add_eos, stack_method=stack_method # instantiate
)

[2024-12-21 02:24:44.116160] Loading vocab from ./data/OGB/pcqm4m-v2/vocab512_stacked ...
[2024-12-21 02:24:44.118654]
{   '0': 22,
    '1': 23,
    '10': 32,
    '100': 122,
    '101': 123,
    '102': 124,
    '103': 125,
    '104': 126,
    '105': 127,
    '106': 128,
    '107': 129,
    '108': 130,
    '109': 131,
    '11': 33,
    '110': 132,
    '111': 133,
    '112': 134,
    '113': 135,
    '114': 136,
    '115': 137,
    '116': 138,
    '117': 139,
    '118': 140,
    '119': 141,
    '12': 34,
    '120': 142,
    '121': 143,
    '122': 144,
    '123': 145,
    '124': 146,
    '125': 147,
    '126': 148,
    '127': 149,
    '128': 150,
    '129': 151,
    '13': 35,
    '130': 152,
    '131': 153,
    '132': 154,
    '133': 155,
    '134': 156,
    '135': 157,
    '136': 158,
    '137': 159,
    '138': 160,
    '139': 161,
    '14': 36,
    '140': 162,
    '141': 163,
    '142': 164,
    '143': 165,
    '144': 166,
    '145': 167,
    '146': 168,
    '147': 169,
    '148': 170,
 

In [8]:
print(gtokenizer)

## set model

In [9]:
import pickle
with open("./zhang_test/model_config.pkl", "rb") as file:  # "rb" mode for reading binary
    config = pickle.load(file)
# print(config)

In [10]:
# use_deepspeed = True

# # 2.2 create model
# if use_deepspeed:
#     deepspeed.init_distributed(
#         dist_backend="nccl", rank=rank, world_size=world_size
#     )

In [11]:
model = GraphModel(config)


# model.gradient_checkpointing_enable()
# silence the warnings. Please re-enable for inference!
model.config.use_cache = False
print_trainable_parameters(model) # 235368960

NOT Applying dropout in backbone transformer
Next-token-prediction changed to next/masked-13-tokens-prediction!
trainable params: 37751808 || all params: 37751808 || trainable%: 100.0


In [12]:

# 2.21 load from ckp IF provided existing ckp and NOT resume from the ckp
ckp, _ = misc_utils.get_latest_ckp(pretrain_cpt)
print(f"Loading pretrained weights from ckp {ckp}")
try:
    # fn_model = os.path.join(ckp, "../model_ema_best.pt")
    # if not os.path.isfile(fn_model):
    fn_model = os.path.join(ckp, "model.pt")
    stat_dict = torch.load(fn_model)
    stat_dict = {
        (k[7:] if k.startswith("module.") else k): v for k, v in stat_dict.items()
    }
    print(f"[{datetime.now()}] load ckp using torch API from:\n{fn_model}")
except Exception as inst:
    # print(type(inst))
    # print(inst.args)
    print("inar: ", inst)
    from deepspeed.utils.zero_to_fp32 import (
        get_fp32_state_dict_from_zero_checkpoint,
    )
    stat_dict = get_fp32_state_dict_from_zero_checkpoint(ckp)
    print(
        f"[{datetime.now()}] load ckp using DeepSpeed API `get_fp32_state_dict_from_zero_checkpoint`"
    )

for key in list(stat_dict.keys()):
    if ("score" in key) and skip_keys:
        stat_dict.pop(key)
        print(f"pop key {key} in stat_dict!")
missing_keys, unexpected_keys = model.load_state_dict(stat_dict, strict=True)
print(
    f"[{datetime.now()}] init model params using pytorch `load_state_dict`\n"
    f"missing keys: {missing_keys}\n"
    f"unexpected_keys: {unexpected_keys}\n"
    f"After loading weights from ckp:\n{model.config}\nmodel-type: {model.dtype}\n\n{model}"
)

Loading pretrained weights from ckp /datalake/datastore1/yang/graph-gpt/exp/models/pcqm4m-v2/medium_ntp/pt_ns_h512_l8_b8192_mpe1024_tk1e9_gelu_pretrain3.3m_nmlm_mrlinear_mtp0.8_0_0.2_lr3e-4_adp0.1_pdp0_edp0_mdp0_lsi0_short_gated_wd0.1/epoch_51
inar:  [Errno 2] No such file or directory: '/datalake/datastore1/yang/graph-gpt/exp/models/pcqm4m-v2/medium_ntp/pt_ns_h512_l8_b8192_mpe1024_tk1e9_gelu_pretrain3.3m_nmlm_mrlinear_mtp0.8_0_0.2_lr3e-4_adp0.1_pdp0_edp0_mdp0_lsi0_short_gated_wd0.1/epoch_51/model.pt'
Processing zero checkpoint '/datalake/datastore1/yang/graph-gpt/exp/models/pcqm4m-v2/medium_ntp/pt_ns_h512_l8_b8192_mpe1024_tk1e9_gelu_pretrain3.3m_nmlm_mrlinear_mtp0.8_0_0.2_lr3e-4_adp0.1_pdp0_edp0_mdp0_lsi0_short_gated_wd0.1/epoch_51/global_step48830'
Detected checkpoint of type zero stage 2, world_size: 1
Parsing checkpoint created by deepspeed==0.15.1
Reconstructed fp32 state dict with 77 params 37751808 elements
[2024-12-21 02:24:45.703601] load ckp using DeepSpeed API `get_fp32_stat

## test

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"device: {device}")
print(model.device)

device: cuda
cuda:0


In [14]:
print(embed_dim)

0


In [15]:
# 1.2 get graph dataset
train_dataset, valid_dataset, test_dataset, raw_dataset = read_dataset(
    name=tokenizer_config["dataset"],   # PCQM4Mv2
    # for local data file reading
    data_dir=data_dir,   # './data/OGB'
    sampling_config=tokenizer_config["sampling"],    # None
    # for odps data reading
    table=tables,   # ""
    edge_dim=tokenizer_config["semantics"]["edge"]["dim"],    # 3
    node_dim=tokenizer_config["semantics"]["node"]["dim"],    # 9
    mode="train",
    # general
    # pretrain_mode=True,
    return_valid_test=True,
    ensemble_datasets=tokenizer_config.get("ensemble_datasets", []),    # []
)
reset_samples_per_epoch = (   # what is this  # None for PCQM4Mv2
    test_dataset.reset_samples_per_epoch
    if hasattr(test_dataset, "reset_samples_per_epoch")
    else False
)
if isinstance(test_dataset, IterableDataset):
    print(next(iter(test_dataset))) 
else: # True
    idx = test_dataset.sampler[0] # (0, Data(edge_index=[2, 40], edge_attr=[40, 3], x=[18, 9], y=[1, 1], num_nodes=18, idx=0, idx_of_ds=0))
    print(test_dataset[idx])


Loading dataset PCQM4Mv2 ...



dataset._data -> Data(edge_index=[2, 109093626], edge_attr=[109093626, 3], x=[52970652, 9], y=[3746620])

Raw indices: 3378606, Removed indices: 0, New indices: 3378606

Raw indices: 73545, Removed indices: 0, New indices: 73545
Using all valid data as valid: 73545, and last half of valid data as test: 36773!
[Warning] permute_nodes enabled! edge_attr remains the same; edge_index and node-attrs will be affected!
[Warning] permute_nodes enabled! edge_attr remains the same; edge_index and node-attrs will be affected!
[Warning] permute_nodes enabled! edge_attr remains the same; edge_index and node-attrs will be affected!
[Warning] permute_nodes enabled! edge_attr remains the same; edge_index and node-attrs will be affected!
[Warning] permute_nodes enabled! edge_attr remains the same; edge_index and node-attrs will be affected!

[2024-12-21 02:24:53.845565] NOT RESET samples of GraphsMapDataset of 3378606 graphs for epoch None!
idx_tuple: None
[Warning] permute_nodes enabled! edge_attr re

In [16]:
print(test_dataset)
print('length of test dataset:', len(test_dataset), 'length of train dataset:', len(train_dataset), 'length of valid dataset:', len(valid_dataset))
print(test_dataset[1])
print(test_dataset[2])
print("#" * 100)
print("example")
print("edge_index: ", test_dataset[1][1].edge_index)
print("edge_attr: ", test_dataset[1][1].edge_attr)
print("x: ", test_dataset[1][1].x)
print("y: ", test_dataset[1][1].y) 

length of test dataset: 36773 length of train dataset: 3378606 length of valid dataset: 73545
(1, Data(edge_index=[2, 34], edge_attr=[34, 3], x=[17, 9], y=[1], num_nodes=17, idx=1, idx_of_ds=0))
(2, Data(edge_index=[2, 32], edge_attr=[32, 3], x=[16, 9], y=[1], num_nodes=16, idx=2, idx_of_ds=0))
####################################################################################################
example
edge_index:  tensor([[ 6,  2,  2, 16, 16, 13, 13, 10, 10,  0,  0,  5, 10,  8,  8, 11, 11,  3,
          3, 12, 12,  7,  7,  9,  9,  1,  1,  4,  1, 15,  9, 14,  3, 16],
        [ 2,  6, 16,  2, 13, 16, 10, 13,  0, 10,  5,  0,  8, 10, 11,  8,  3, 11,
         12,  3,  7, 12,  9,  7,  1,  9,  4,  1, 15,  1, 14,  9, 16,  3]])
edge_attr:  tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 1],
        [0, 0, 1],
        [3, 0, 1],
        [3, 0, 1],
        [3, 0, 1],
        [3, 0, 1],
        [0, 0, 1],
        [0, 0, 1],
        [0, 0, 0],
        [0, 0, 0],
        [3, 0, 1],
        [3, 

In [17]:
graph_test = test_dataset[2][1]
from src.utils.my_utiles import graph2smiles
smiles = graph2smiles(graph_test.edge_index, graph_test.edge_attr, graph_test.x)
print(smiles)

C=CCN(C=Cc1ccccc1C)C(C)=O


In [18]:
from src.utils.my_utiles import smiles2graph
smiles = "CCO"
graph = smiles2graph("CC(=O)OC1=CC=CC=C1C(=O)O")
print(graph.keys())
print(graph['edge_index'].shape)
print(graph['edge_attr'].shape)
print(graph['x'].shape)
print(graph['num_nodes'])


dict_keys(['edge_index', 'edge_attr', 'x', 'num_nodes'])
(2, 26)
(26, 3)
(13, 9)
13


In [ ]:
from src.utils.my_utiles import graph_to_torch_geometric
graph = graph_to_torch_geometric(graph)
print(graph['edge_index'].shape)
print(graph['edge_attr'].shape)
print(graph['x'].shape)
print(graph['num_nodes'])

torch.Size([2, 26])
torch.Size([26, 3])
torch.Size([13, 9])
13


In [ ]:
from src.utils.my_utiles import graph2smiles
smiles = graph2smiles(graph.edge_index, graph.edge_attr, graph.x)
print(smiles)

CC(=O)Oc1ccccc1C(=O)O


In [ ]:
example_graph = graph_test
from src.utils.my_utiles import graph2token2input
import numpy as np
token, label, embed, inputs = graph2token2input(example_graph, gtokenizer)

print(
    f"\nTokens:\n{pformat(token)}\nLabels:\n{pformat(label)}\nembed:{np.array(embed)}\n"
)

print(f"Inputs for model:\n{pformat(inputs)}\n")

Inspecting tokenization results!
Tokenize graph:
Data(edge_index=[2, 32], edge_attr=[32, 3], x=[16, 9], y=[1], num_nodes=16, idx=2, idx_of_ds=0)
[Warning] Set eos_idx to 100000000 for task pretrain!

Tokens:
[['50',
  'molecule#node#0#5',
  'molecule#node#1#0',
  'molecule#node#2#3',
  'molecule#node#3#5',
  'molecule#node#4#1',
  'molecule#node#5#0',
  'molecule#node#6#1',
  'molecule#node#7#0',
  'molecule#node#8#0',
  'molecule#edge#0',
  'molecule#edge#1',
  'molecule#edge#2'],
 ['51',
  'molecule#node#0#6',
  'molecule#node#1#0',
  'molecule#node#2#3',
  'molecule#node#3#5',
  'molecule#node#4#0',
  'molecule#node#5#0',
  'molecule#node#6#1',
  'molecule#node#7#0',
  'molecule#node#8#0',
  'molecule#edge#0#0',
  'molecule#edge#1#0',
  'molecule#edge#2#1'],
 ['52',
  'molecule#node#0#5',
  'molecule#node#1#0',
  'molecule#node#2#4',
  'molecule#node#3#5',
  'molecule#node#4#2',
  'molecule#node#5#0',
  'molecule#node#6#2',
  'molecule#node#7#0',
  'molecule#node#8#0',
  'molecule#e

In [22]:
print(inputs.keys())

dict_keys(['input_ids', 'position_ids', 'labels', 'attention_mask', 'embed'])


In [23]:
from src.utils.my_utiles import convert_to_tensors

tensor_inputs = convert_to_tensors(inputs)
print(tensor_inputs.keys())

print("input_ids:", tensor_inputs["input_ids"].shape)
print("position_ids:", tensor_inputs["position_ids"].shape)
print("labels:", tensor_inputs["labels"].shape)
print("attention_mask:", tensor_inputs["attention_mask"].shape)
print("embed:", tensor_inputs["embed"].shape)


dict_keys(['input_ids', 'position_ids', 'labels', 'attention_mask', 'embed'])
input_ids: torch.Size([1, 25, 13])
position_ids: torch.Size([1, 25])
labels: torch.Size([1, 25, 13])
attention_mask: torch.Size([1, 25])
embed: torch.Size([1, 25, 0])


In [ ]:
print(tensor_inputs["input_ids"])
print(tensor_inputs["labels"])

tensor([[[ 72, 630, 685, 695, 709, 715, 724, 731, 736, 738, 740, 741, 742],
         [ 73, 641, 685, 695, 709, 714, 724, 731, 736, 738, 743, 748, 755],
         [ 74, 630, 685, 696, 709, 716, 724, 732, 736, 738, 743, 748, 754],
         [ 75, 630, 685, 695, 709, 715, 724, 731, 736, 738, 743, 748, 754],
         [ 76, 630, 685, 695, 709, 716, 724, 731, 736, 738, 744, 748, 754],
         [ 75, 630, 685, 695, 709, 715, 724, 731, 736, 738, 744, 748, 754],
         [ 74, 630, 685, 696, 709, 716, 724, 732, 736, 738, 743, 748, 754],
         [ 73, 641, 685, 695, 709, 714, 724, 731, 736, 738, 743, 748, 754],
         [ 77, 630, 685, 695, 709, 714, 724, 731, 736, 738, 743, 748, 755],
         [ 78, 652, 685, 691, 709, 714, 724, 731, 736, 738, 744, 748, 755],
         [ 77, 630, 685, 695, 709, 714, 724, 731, 736, 738, 744, 748, 755],
         [ 79, 630, 685, 696, 709, 717, 724, 732, 736, 738, 743, 748, 754],
         [ 77, 630, 685, 695, 709, 714, 724, 731, 736, 738, 743, 748, 754],
         [ 7

In [25]:
print("input_ids:", tensor_inputs["input_ids"].shape)
print("position_ids:", tensor_inputs["position_ids"].shape)
print("labels:", tensor_inputs["labels"].shape)
print("attention_mask:", tensor_inputs["attention_mask"].shape)
print("embed:", tensor_inputs["embed"].shape)


input_ids: torch.Size([1, 25, 13])
position_ids: torch.Size([1, 25])
labels: torch.Size([1, 25, 13])
attention_mask: torch.Size([1, 25])
embed: torch.Size([1, 25, 0])


In [26]:
tensor_inputs["embed"]

tensor([], size=(1, 25, 0), dtype=torch.float64)

In [27]:
from src.utils.my_utiles import graph2token2input_generation
# smiles = "CCO" # "CC(=O)OC1=CC=CC=C1C(=O)O"\
random_index = np.random.randint(0, len(test_dataset))  # Upper bound is exclusive
print(random_index)
random_index = 0
graph = test_dataset[random_index][1]
# graph = graph_to_torch_geometric(graph)
smiles2 = graph2smiles(graph.edge_index, graph.edge_attr, graph.x)
example_graph = graph
print(smiles2)

num_input, max_length = 5, 40
token, label, embed, inputs = graph2token2input(graph, gtokenizer)
print('token', np.array(token).shape)
print('label', np.array(label).shape)
print('embed', np.array(embed).shape)
print(inputs.keys())
print('input_ids', np.array(inputs["input_ids"]).shape)
print('position_ids', np.array(inputs["position_ids"]).shape)
print('labels', np.array(inputs["labels"]).shape)
print('attention_mask', np.array(inputs["attention_mask"]).shape)
# print('embed', np.array(inputs["embed"]).shape)
print(inputs["attention_mask"])
tensor_inputs = convert_to_tensors(inputs)

28754
Cc1ccc(C2Cc3cnccc3NC2=O)cc1
Inspecting tokenization results!
Tokenize graph:
Data(edge_index=[2, 40], edge_attr=[40, 3], x=[18, 9], y=[1], num_nodes=18, idx=0, idx_of_ds=0)
token (25, 13)
label (25, 13)
embed (25, 0)
dict_keys(['input_ids', 'position_ids', 'labels', 'attention_mask', 'embed'])
input_ids (25, 13)
position_ids (25,)
labels (25, 13)
attention_mask (25,)
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [28]:
model.eval()
data = tensor_inputs
input_ids = data["input_ids"].to(device)
# print(input_ids)
attention_mask = data["attention_mask"].to(device)
labels = data["labels"].to(device)
inputs_raw_embeds = None
if embed_dim > 0: # in tokenizer config
    inputs_raw_embeds = data["embed"].to(device)
print(inputs_raw_embeds)
output = model(
    input_ids=input_ids,
    attention_mask=attention_mask,
    labels=labels,
    inputs_raw_embeds=inputs_raw_embeds,
)  # Perform a single forward pass.
print(output)


None
begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---
input_embeds None
input_ids at beginning torch.Size([1, 25, 13])
inputs_embeds torch.Size([1, 25, 512])
attention_mask torch.Size([1, 25])
odict_keys(['last_hidden_state'])
torch.Size([1, 25, 512])
labels torch.Size([1, 25, 13])
labels_m torch.

In [29]:
# print("labels:", tensor_inputs["labels"].shape)
# print("output:", output.keys())
# print("output:", output["logits"].shape)

In [30]:
# predicted_labels = torch.argmax(output["logits"], dim=-1) 
# reshaped_labels = predicted_labels.view(*tensor_inputs["labels"].shape)
# print("labels:", tensor_inputs["labels"])
# print("predicted_labels:", reshaped_labels)

In [31]:
# # File path to your vocabulary file
# vocab_file_path = "/datalake/datastore1/yang/graph-gpt/data/OGB/pcqm4m-v2/vocab512_stacked"

# from src.utils.my_utiles import load_vocab

# vocab = load_vocab(vocab_file_path)

# from src.utils.my_utiles import convert_labels_to_tokens
# # Example usage
# # Assuming `reshaped_labels` contains the predicted label IDs of shape [1, 24, 13]
# tokens = convert_labels_to_tokens(reshaped_labels, vocab)

# # Optional: Reshape tokens back to the original structure for visualization
# tokens_reshaped = [
#     [tokens[i * 13 + j] for j in range(13)] for i in range(tensor_inputs["labels"].shape[1])
# ]

# print(reshaped_labels.shape)
# # Print tokens
# print(np.array(tokens_reshaped).shape)
# pprint(tokens_reshaped)


# test generation

In [ ]:
from src.utils.my_utiles import graph2token2input_generation
# smiles = "CCO" # "CC(=O)OC1=CC=CC=C1C(=O)O"\
random_index = np.random.randint(0, len(test_dataset))  # Upper bound is exclusive
print(random_index)
random_index = 0
graph = test_dataset[random_index][1]
# graph = graph_to_torch_geometric(graph)
smiles2 = graph2smiles(graph.edge_index, graph.edge_attr, graph.x)
example_graph = graph
print(smiles2)

num_input, max_length = 5, 40
token, label, embed, inputs = graph2token2input_generation(graph, gtokenizer, num_input, max_length)
print('token', np.array(token).shape)
print('label', np.array(label).shape)
print('embed', np.array(embed).shape)
print(inputs.keys())
print('input_ids', np.array(inputs["input_ids"]).shape)
print('position_ids', np.array(inputs["position_ids"]).shape)
print('labels', np.array(inputs["labels"]).shape)
print('attention_mask', np.array(inputs["attention_mask"]).shape)
# print('embed', np.array(inputs["embed"]).shape)
print(inputs["attention_mask"])

29082
Cc1ccc(C2Cc3cnccc3NC2=O)cc1
Inspecting tokenization results!
Tokenize graph:
Data(edge_index=[2, 40], edge_attr=[40, 3], x=[18, 9], y=[1], num_nodes=18, idx=0, idx_of_ds=0)
token (5, 13)
label (25, 13)
embed (5, 0)
dict_keys(['input_ids', 'position_ids', 'labels', 'attention_mask', 'embed'])
input_ids (5, 13)
position_ids (5,)
labels (25, 13)
attention_mask (5,)
[1. 1. 1. 1. 1.]


In [33]:
# pprint(token)
# pprint(label)
# pprint(inputs["input_ids"])
# pprint(inputs["labels"])

In [34]:
tensor_inputs = convert_to_tensors(inputs)
# print(tensor_inputs)

In [35]:
model.device

device(type='cuda', index=0)

In [ ]:
tensor_inputs.keys()

dict_keys(['input_ids', 'position_ids', 'labels', 'attention_mask', 'embed'])

In [37]:
model.eval()
output_ids = model.generate(
    input_ids=tensor_inputs["input_ids"].to(device) ,          # Input token IDs
    attention_mask=tensor_inputs["attention_mask"].to(device),  # Pass the attention mask
    position_ids=tensor_inputs["position_ids"].to(device),      # Pass the position IDs
    max_length=20,                # Maximum generation length
    do_sample=True,               # Enable sampling
    temperature=0.7               # Temperature for randomness
)

begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---begin model ---
input_embeds None
input_ids at beginning torch.Size([1, 5, 13])
inputs_embeds torch.Size([1, 5, 512])
attention_mask torch.Size([1, 5])
odict_keys(['last_hidden_state', 'past_key_values'])
torch.Size([1, 5, 512])
tensor([[True, True, True, True, Tru

AttributeError: 'tuple' object has no attribute 'shape'